In [1]:
import sys
sys.path.append("/home/mat/repos/gpu_monitor") 

In [2]:
from gpumon import nvidia_run, extract_and_print, extract
import asyncio
%load_ext autoreload
%autoreload 2

In [3]:
%%time
a=extract(nvidia_run())

CPU times: user 8 ms, sys: 4 ms, total: 12 ms
Wall time: 1.59 s


In [4]:
async def fetch():
    return extract(nvidia_run())

In [5]:
async def produce(sem, queue):
    while True:
        # produce an item
        message = await fetch(sem)
        await asyncio.sleep(0)
        # put the item in the queue
        await queue.put(message)

In [6]:
async def consume(queue):
    while True:
        # wait for an item from the producer
        res = await queue.get()
        print(res)
#         fail_probability = float(res['message'])
#         await update_plot(index, fail_probability)

        # Notify the queue that the item has been processed
        queue.task_done()

In [7]:
async def run():
    queue = asyncio.Queue()
    # create instance of Semaphore
    sem = asyncio.Semaphore(10)

    # Create client session that will ensure we dont open new connection
    # per each request.
    # schedule the consumer
    consumer = asyncio.ensure_future(consume(queue))
    await produce(sem, queue)
        
    # wait until the consumer has processed all items
    await queue.join()
    # the consumer is still awaiting for an item, cancel it
    consumer.cancel()

In [8]:
try:
    loop = asyncio.get_event_loop()
    loop.run_until_complete(run())
finally:
    print('Shutdown')
    loop.close()


Shutdown


TypeError: fetch() takes 0 positional arguments but 1 was given